In [1]:
import pandas as pd
from pathlib import Path
from datetime import date,datetime


current_path = Path.cwd()
csv_name = "investimentos_movimentacoes.csv"
raw_data = pd.read_csv(filepath_or_buffer=current_path/csv_name)
list_to_pop = ["Lucro","Imposto","Observação","F sup","Preço médio","Valor total"]
for name in list_to_pop:
    raw_data.pop(name)
    



In [2]:
class Portfolio:
    def __init__ (self,data_base):
        self.data:pd.DataFrame=data_base

    def get_values(self,dict_values_to_filter) -> pd.DataFrame:

        mask = self.data["buy"]==self.data["buy"]
        for column,value in dict_values_to_filter.items():
            mask = (self.data[column]==value) & mask
        return self.data[mask]

In [3]:
data_base = pd.DataFrame()
data_base["owner"] = raw_data["Banco"].apply(lambda text: "Luiz" if text =="Inter" else "Luciane")
data_base["date"] = raw_data["Data operação"].apply(lambda date: datetime.strptime(date,"%d/%m/%Y"))
data_base["ticker"] = raw_data["Nome ativo"]
data_base["buy"] = raw_data["C/V"].apply(lambda text: True if text=="C" else False)
data_base["price"] = raw_data["Preço operação"].apply(lambda price_text: float(''.join(filter(lambda x: x.isdigit() or x in ',', price_text)).replace(",",".")))
data_base["amount"] = raw_data["Qnt"].apply(lambda Qnt: float(Qnt.replace(",",".")))
data_base["curency"] = "BRL"

In [5]:
display(data_base["ticker"].unique())

array(['ALZR11', 'LFTS11', 'SLCE3', 'SPXI11', 'Prefixado 2029', 'HGRU11',
       'HSML11', 'SUZB3', 'BBAS3', 'BIL', 'IBIT', 'ROMI3', 'HGPO11',
       'ITSA4', 'Selic 2029', 'CMIG4', 'GGBR4', 'BBDC4', 'HGLG11',
       'BODB11', 'VALE3', 'TRXF11', 'CLIN11', 'CPTI11', 'IPCA+ 2045',
       'TRPL4', 'KNSC11', 'KNIP11', 'RBRR11', 'TLT', 'CIEL3', 'ALUG11',
       'BITH11', 'SAPR4', 'KORE11', 'HGRE11', 'RBVA11', 'KNCA11', 'PSSA3',
       'BTCR11', 'JSRE11', 'CVBI11', 'BCRI11', 'GGRC11', 'HGCR11',
       'KNCR11', 'USDB11', 'Selic 2026', 'IPCA+ 2029', 'SHY', 'IVV'],
      dtype=object)

In [8]:
_list_profiles = self.data["owner"].unique().tolist()
_list_profiles = ["Luiz"]

for profile in _list_profiles:
        
    _position_dataframe= pd.DataFrame()
    _amounts = []
    _actual_prices = []
   
    for ticker in self.get_values({"owner":profile})["ticker"].unique():
        _amount = (sum(self.get_values({"owner":profile,"ticker":ticker,"buy":True})["amount"]) 
                 - sum(self.get_values({"owner":profile,"ticker":ticker,"buy":False})["amount"]))  
        _position_dataframe.loc[ticker]["Quantidade"] = _amount
    
        # _position_dataframe["Valor atual"][ticker] = _actual_price
    display(_position_dataframe)

NameError: name 'self' is not defined

In [38]:
import investiny as invest
portifolio = Portfolio(data_base)
self = portifolio
display(portifolio.data["ticker"].unique())
assets = []
for name_asset in portifolio.data["ticker"].unique():
    _temp = invest.search_assets(query=name_asset, limit=1)
    assets.append(_temp[0]["ticker"])
    display(_temp)


array(['ALZR11', 'LFTS11', 'SLCE3', 'SPXI11', 'Prefixado 2029', 'HGRU11',
       'HSML11', 'SUZB3', 'BBAS3', 'BIL', 'IBIT', 'ROMI3', 'HGPO11',
       'ITSA4', 'Selic 2027', 'CMIG4', 'GGBR4', 'BBDC4', 'HGLG11',
       'BODB11', 'VALE3', 'TRXF11', 'CLIN11', 'CPTI11', 'IPCA+ 2045',
       'TRPL4', 'KNSC11', 'KNIP11', 'RBRR11', 'TLT', 'CIEL3', 'ALUG11',
       'BITH11', 'SAPR4', 'KORE11', 'HGRE11', 'RBVA11', 'KNCA11', 'PSSA3',
       'BTCR11', 'JSRE11', 'CVBI11', 'BCRI11', 'GGRC11', 'HGCR11',
       'KNCR11', 'USDB11', 'IPCA+ 2029', 'SHY', 'IVV'], dtype=object)

[{'symbol': 'ALZR11',
  'full_name': 'B3:ALZR11',
  'description': 'Alianza Trust Renda Imobiliaria FII',
  'type': 'Stock',
  'ticker': '1059885',
  'exchange': 'B3'}]

[{'symbol': 'LFTS11',
  'full_name': 'B3:LFTS11',
  'description': 'INVESTO TEVA TESOURO SELIC ETF - FUNDO DE INVESTIMENTO DE ÍNDICE',
  'type': 'ETF',
  'ticker': '1198214',
  'exchange': 'B3'}]

[{'symbol': 'SLCE3',
  'full_name': 'B3:SLCE3',
  'description': 'SLC Agricola SA',
  'type': 'Stock',
  'ticker': '18781',
  'exchange': 'B3'}]

[{'symbol': 'SPXI11',
  'full_name': 'B3:SPXI11',
  'description': 'It Now S&P500 TRN',
  'type': 'ETF',
  'ticker': '956434',
  'exchange': 'B3'}]

...

[{'symbol': 'HGRE11',
  'full_name': 'B3:HGRE11',
  'description': 'FII HG REAL',
  'type': 'Stock',
  'ticker': '940954',
  'exchange': 'B3'}]

[{'symbol': 'RBVA11',
  'full_name': 'B3:RBVA11',
  'description': 'FII Rio Bravo Rendo Varejo',
  'type': 'Stock',
  'ticker': '1131594',
  'exchange': 'B3'}]

[{'symbol': 'KNCA11',
  'full_name': 'B3:KNCA11',
  'description': 'Kinea Crédito Agro Fiagro Imobiliário',
  'type': 'Stock',
  'ticker': '1185702',
  'exchange': 'B3'}]

[{'symbol': 'PSSA3',
  'full_name': 'B3:PSSA3',
  'description': 'Porto Seguro SA',
  'type': 'Stock',
  'ticker': '18760',
  'exchange': 'B3'}]

IndexError: list index out of range

In [12]:
display(assets)

NameError: name 'assets' is not defined

In [6]:
display(portifolio.data)

,owner,date,ticker,buy,price,amount,curency
0,Luciane,2024-06-07,ALZR11,True,107.43,2.0,USD
1,Luiz,2024-06-07,ALZR11,True,107.43,1.0,USD
2,Luiz,2024-06-07,LFTS11,True,120.19,8.0,USD
3,Luciane,2024-05-28,LFTS11,True,119.86,40.0,USD
4,Luciane,2024-05-20,LFTS11,False,119.59,42.0,USD
...,...,...,...,...,...,...,...
389,Luiz,2020-12-11,SPXI11,True,195.89,1.0,USD
390,Luiz,2020-12-04,SPXI11,True,201.92,1.0,USD
391,Luiz,2020-11-25,SPXI11,True,204.60,1.0,USD
392,Luiz,2020-11-18,SPXI11,True,202.65,1.0,USD


ALZR11.SA: No price data found, symbol may be delisted (1d 2024-06-09 -> 2024-06-09)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [27]:
display(_position_dataframe)

,Quantidade,Preço atual
ALZR11,4.0,109.190002
LFTS11,8.0,120.18
HGRU11,0.0,128.0
SUZB3,24.0,48.650002
BBAS3,48.0,27.18
BIL,NaN,NaN
IBIT,NaN,NaN
ROMI3,NaN,NaN
HGPO11,NaN,NaN
ITSA4,NaN,NaN


In [16]:
display(yf.Ticker("ALZR11.SA").history(start=date.today())["Close"].values)

array([109.19000244])

In [ ]:
list_profiles = data_base["owner"].unique().tolist()
list_profiles = ["Luiz"]
for profile in list_profiles:
    _df = data_base[data_base["owner"]==profile]
    
    _dict_amount = {}
    for ticker in _df["ticker"].unique():

        _ticker_buy_mask = _df[(_df["ticker"]==ticker) and _df["buy"]]
        _ticker_sell_mask = _df[(_df["ticker"]==ticker) and not _df["buy"]]
        _amount = sum(_df[_ticker_buy_mask]["amount"])-sum(_df[_ticker_sell_mask]["amount"])    
        _dict_amount[ticker] = _amount


In [ ]:
startdate = date(2020,1,1)
end_date = date.today()

df = pd.DataFrame()
for ticker in stocksymbols:

    data = yf.Ticker(ticker).history(start=startdate,end=end_date)[["Close"]]
    data.columns = [ticker]

    if np.size(df) == 0:
        df = data
    else:
        df = df.join(data)


In [ ]:
list_transactions = []
for index in raw_data.index:
    list_transactions.append(
        Transaction(
            owner=raw_data.loc[index]["Banco"]
            
        )
        )